In [76]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pickle
import time
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
def tstr():
    '''
    provides a string of current time in PST
    for time sensitive file names with no colons or spaces
    '''
    out = ''
    for i in time.ctime():
        if i!=' ' and i!=':':
            out+=i
        else:
            out+='_'
    return out

In [2]:
catmednan = np.load('/content/drive/MyDrive/Students/catCirrMedNan.npy')

In [3]:
mednan = np.load('/content/drive/MyDrive/Students/cirrhosis_medNaN.npy')

In [86]:
def gen_model():
    model = keras.Sequential([
            layers.Dense(100, activation="relu", 
                    kernel_regularizer=keras.regularizers.L2(3e-2)),
            layers.Dense(4,activation= 'softmax',
                    kernel_regularizer=keras.regularizers.L2(3e-2 ))])
    model.compile(
        loss=keras.losses.CategoricalCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(learning_rate=5e-3),
        metrics=[ 'accuracy'])
    return model

In [20]:
def get_train(x=None, y = None, lst = [1,3,6]):
  outx = []
  outy = []
  for i in lst:
      outx.append(x[i])
      outy.append(y[i])
      
  return np.asarray(outx), np.asarray(outy)
        

In [85]:
def train(train_examples=None,train_labels=None, 
        validation_data = None, model = None, epochs = 2):
  if model==None:
    model = gen_model()
  hist = model.fit(train_examples,train_labels, 
      validation_data = validation_data,epochs = epochs, verbose=0)
  return hist.history['val_loss'][-1]

In [94]:
def tmcDataShapley(x=catmednan[:325,:-4], y = catmednan[:325,-4:],
                   valx = catmednan[325:,:-4], valy= catmednan[325:,-4:], 
                   N = 5, label = ''):
  Ndata, Ddata = x.shape
  results = {}
  for i in range(N):
    delta_loss_lst = [15.0,15.0,15.0]
    loss = 100
    delta_loss=100
    dlosses = []
    losses = []
    lst = []
    model = gen_model()
    while np.sum(np.abs(delta_loss_lst)<7) <len(delta_loss_lst) and len(lst)<Ndata:
      # these two conditionals concern whether there has been recent plateau in
      # learning and whether all samples have been used, both cases lead to
      # a breaking from the loop
      k= np.random.randint(0,Ndata)
      while k in lst:
        k= np.random.randint(0,Ndata)
      lst.append(k)
      if len(lst)%5==0:
        print('running with ',len(lst), ' samples')
      newx,newy = get_train(x,y,lst)
      newloss = train(x,y,(valx,valy), model,4)
      if len(lst)>1:
        if k not in results:
          results[k] = []
        delta_loss = loss-newloss
        if len(dlosses)<4:
          pass
        else:
          delta_loss_lst = dlosses[-3:]

        results[k].append(delta_loss)
      loss = newloss
      losses.append(loss)
      dlosses.append(delta_loss)
      tf.keras.backend.clear_session()
      if len(lst)%20==0:
        print('\nLIST HAS '+str(len(lst))+' SAMPLES\n')
        print('The latest delta loss values are ',str(delta_loss_lst))
        print('\nAnd the latest loss is ',loss,'\n')
    results['dlosses'+str(i)] = dlosses
    results['losses'+str(i)] = losses
    f = open("/content/drive/MyDrive/Students/shapley/David_"
            +label+".pkl","wb")
    pickle.dump(dict,f)
    f.close()

    print('\nSAVED RESULTS for loop\n'+str(i))
  return results

In [72]:
#sample code for loading a dictionary from a pkl file

'''
filename = "/content/drive/MyDrive/Students/shapley/file.pkl"
unpickleFile = open(filename, 'rb')
new_dict = pickle.load(unpickleFile, encoding='latin1')
'''

'\nfilename = "/content/drive/MyDrive/Students/shapley/file.pkl"\nunpickleFile = open(filename, \'rb\')\nnew_dict = pickle.load(unpickleFile, encoding=\'latin1\')\n'

In [95]:
results = tmcDataShapley(label='SavingTests',N=7)

running with  5  samples
running with  10  samples
running with  15  samples
running with  20  samples

LIST HAS 20 SAMPLES

The latest delta loss values are  [49.11157989501953, -96.84654998779297, 89.17662811279297]

And the latest loss is  63.537017822265625 

running with  25  samples
running with  30  samples
running with  35  samples
running with  40  samples

LIST HAS 40 SAMPLES

The latest delta loss values are  [-37.522247314453125, 26.90191650390625, 4.076391220092773]

And the latest loss is  28.09872817993164 

running with  45  samples
running with  50  samples
running with  55  samples

SAVED RESULTS for loop
0
running with  5  samples
running with  10  samples
running with  15  samples
running with  20  samples

LIST HAS 20 SAMPLES

The latest delta loss values are  [6.1366119384765625, -20.205612182617188, -75.96296691894531]

And the latest loss is  32.51509475708008 

running with  25  samples
running with  30  samples
running with  35  samples
running with  40  sampl

In [96]:
results

{0: [36.52042770385742, 15.777259826660156],
 1: [-20.205612182617188],
 10: [-32.52375793457031],
 100: [89.17662811279297],
 104: [-23.944786071777344],
 105: [65.43330383300781, 24.162242889404297],
 106: [-4.101406097412109],
 107: [-88.13224029541016, -23.23845672607422],
 108: [-81.49016571044922, -10.627643585205078, 17.212974548339844],
 109: [-24.594451904296875],
 11: [-29.83673095703125],
 110: [26.711979866027832],
 111: [40.73892021179199],
 112: [10.082291603088379],
 113: [8.638635635375977, -3.9487075805664062, 40.809051513671875],
 114: [13.925352096557617],
 115: [33.7044792175293, -62.207868576049805],
 116: [-10.965121269226074,
  -48.29668045043945,
  -36.63637924194336,
  -23.520950317382812],
 118: [-10.473682403564453,
  90.30402374267578,
  26.95913314819336,
  -52.8887825012207],
 119: [-36.86311340332031],
 12: [103.97222137451172, -14.052447319030762],
 120: [10.863739013671875, 94.17558860778809],
 121: [-98.14370727539062, 51.095916748046875],
 122: [-70.3

In [ ]:
rmses = {}

EPOCHS = 30
L2REG = 1e-2
LR = 1e-3
SAVE = False
for L2REG in [1e-2, 3e-2,1e-1]:
  for LR in [3e-4,1e-3,3e-3]:
    LABEL = ('100_LR:'+str(LR)+'_REG:'+str(L2REG)+
            '_EPOCH:'+str(EPOCHS))
    
    print(
      '\n\n\n\nRunning '+LABEL+' experiment'
    )
    model = keras.Sequential([
            layers.Dense(100, activation="relu", 
                    kernel_regularizer=keras.regularizers.L2(L2REG )),
            layers.Dense(4,activation= 'softmax',
                    kernel_regularizer=keras.regularizers.L2(L2REG ))])
    model.compile(
        loss=keras.losses.CategoricalCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
        metrics=[ 'accuracy'])
    hist = model.fit(catmednan[:325,:-4],catmednan[:325,-4:],
                    epochs=EPOCHS,
                    validation_data=(catmednan[325:,:-4],
                                      catmednan[325:,-4:]))
    
    acc = hist.history['accuracy'][-1]

    pred= model.predict(catmednan[:,:-4])
    curvepred = pred @ np.array([[1],[2],[3],[4]])
    curvermse = np.sqrt(np.mean(np.square(mednan[:,-1]-curvepred)))

    rmses[LABEL] = curvermse

    if acc > .45:
      hist.history['rmse'] = [curvermse]*EPOCHS
      histdf = pd.DataFrame(hist.history)
      histdf.to_csv('/content/drive/MyDrive/Students/models/DavidTests'+tstr()[4:-8]+LABEL+'.csv')
      model.save('/content/drive/MyDrive/Students/models/DavidTests'+tstr()[4:-8]+LABEL)

rmseDF = pd.DataFrame(rmses, index=['rmse'])
rmseDF.to_csv('/content/drive/MyDrive/Students/records/DavidTests'+tstr()[4:-8]+'.csv')